# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`MET-Data`*

---
*@author: Evan*\
*@date: 2023-06-06*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.io.shapereader import Reader

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
# uvwind = xr.open_dataset('F:/Data/Project_anqing/May/COMBINE_ACONC_v54_intel_CN3AH_135X138_2023_wind.nc')
met = xr.open_dataset('F:/Data/Project_anqing/May/COMBINE_ACONC_v54_intel_CN3AH_135X138_2023_mcip-2.nc')
grid = xr.open_dataset('F:/Data/Project_anqing/GRIDCRO2D_2023021.nc')

In [3]:
# 将时间点转换为实际时间
times=pd.date_range('2023-05-01-00','2023-05-31-23',freq='h')
times

DatetimeIndex(['2023-05-01 00:00:00', '2023-05-01 01:00:00',
               '2023-05-01 02:00:00', '2023-05-01 03:00:00',
               '2023-05-01 04:00:00', '2023-05-01 05:00:00',
               '2023-05-01 06:00:00', '2023-05-01 07:00:00',
               '2023-05-01 08:00:00', '2023-05-01 09:00:00',
               ...
               '2023-05-31 14:00:00', '2023-05-31 15:00:00',
               '2023-05-31 16:00:00', '2023-05-31 17:00:00',
               '2023-05-31 18:00:00', '2023-05-31 19:00:00',
               '2023-05-31 20:00:00', '2023-05-31 21:00:00',
               '2023-05-31 22:00:00', '2023-05-31 23:00:00'],
              dtype='datetime64[ns]', length=744, freq='H')

In [4]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [5]:
# calculate saturation vapor pressure (es)
es = 6.112 * np.exp((17.67 * met.SFC_TMP) / (met.SFC_TMP + 243.5))

# calculate vapor pressure (e)
e = met.PRES * met.QV / (0.622 + 0.378 * met.QV)

# calculate relative humidity (RH)
RH = e / es * 100

In [6]:
days=1 # set spin-up days
dataset=xr.Dataset(
    data_vars=dict(
        # ! vars from mcip
        RH=(['time','level','y','x'],RH[days*24-8:-8-1,:21,:,:].data,{'long name':'Relative Humidity on Surface','units':'%'}),
        PRES=(['time','level','y','x'],met.PRES[days*24-8:-8-1,:21,:,:].data,{'long name':'Air Pressure','units':'hPa'}),
        ),
    coords=dict(
        time=times,
        level=pres[:21],
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202305',
        grid='CN3AH_135X138',
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (time: 744, level: 21, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-05-01 ... 2023-05-31T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 859.5 833.8 804.0
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    RH         (time, level, y, x) float32 52.72 52.54 57.14 ... 0.0 0.0 0.0
    PRES       (time, level, y, x) float32 972.1 942.3 919.6 ... 790.0 790.1
Attributes:
    case:     Anqing_202305
    grid:     CN3AH_135X138

In [7]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset.data_vars}
dataset.to_netcdf('D:/Download/May_presrh.nc',encoding=encoding)